In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [35]:
data = pd.read_csv('Churn_Modelling.csv')

In [36]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [37]:
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'],axis=1)

In [38]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [39]:
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [40]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder()
dm = one_hot_encoder.fit_transform(data[['Geography']])
dm = pd.DataFrame(dm.toarray(), columns=one_hot_encoder.get_feature_names_out(['Geography']))
dm.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [41]:
data_updated = pd.concat([data, dm], axis=1)
data_updated.drop(columns=['Geography'], axis=1, inplace=True)
data_updated.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,Female,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,Female,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [42]:
label_encoder = LabelEncoder()
data_updated['Gender'] = label_encoder.fit_transform(data_updated['Gender'])

In [43]:
data_updated.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [44]:
## save encoders and scaler
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

with open('one_hot_encoder.pkl', 'wb') as f:
    pickle.dump(one_hot_encoder, f)

In [47]:
# divide into train and test
X = data_updated.drop(columns=['Exited'], axis=1)
y = data_updated['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaler features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [48]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [49]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [60]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [63]:
# Build ANN Model

model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32 , activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [64]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                832       
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [68]:
import datetime
# Setup TensorBoard
tensorboard = TensorBoard(log_dir=f'logs/fit/{datetime.time().strftime("%Y%m%d-%H%M%S")}', histogram_freq=1)

## Setup Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[tensorboard, early_stopping])

Epoch 1/100


250/250 [==============================] - 2s 5ms/step - loss: 0.4540 - accuracy: 0.7986 - val_loss: 0.3917 - val_accuracy: 0.8305
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3856 - accuracy: 0.8446 - val_loss: 0.3526 - val_accuracy: 0.8525
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3555 - accuracy: 0.8581 - val_loss: 0.3414 - val_accuracy: 0.8540
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3441 - accuracy: 0.8605 - val_loss: 0.3394 - val_accuracy: 0.8625
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3389 - accuracy: 0.8611 - val_loss: 0.3386 - val_accuracy: 0.8630
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3349 - accuracy: 0.8624 - val_loss: 0.3441 - val_accuracy: 0.8615
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3326 - accuracy: 0.8616 - val_loss: 0.3385 - val_accuracy: 0.86

In [69]:
model.save('model.h5')

c:\Users\ayush_hakhu\Downloads\code\gen_ai\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [70]:
# lOAD Tensorboard
%load_ext tensorboard

In [72]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 45820), started 0:00:39 ago. (Use '!kill 45820' to kill it.)